In [91]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)       # Increase width of the display
pd.set_option('display.colheader_justify', 'center')  # Center align column headers

In [92]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import display, clear_output


In [93]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


Torch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
Device name: NVIDIA GeForce RTX 4060 Laptop GPU


### LLM Model Downloading

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.3"

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)


### Preparing the Data

In [94]:
import joblib
product_labeled_df = joblib.load('./Data/product_insights.joblib')
label_details = joblib.load('./Data/label_details.joblib')

In [95]:
product_labeled_df = product_labeled_df.reset_index()

In [96]:
product_labeled_df.head()


,Product_Name,Count_of_Purchases,Total_Quantity,Total_Revenue,Repeat_Purchase_Rate,Days_Since_Last_Purchase,most_used_payment_type,sales_performance_segmentation,customer_engagement_segmentation,purchase_timing_segmentation,payment_behavior_segmentation,product_segmentation,geographical_segmentation,Hierarchy_METIER,Hierarchy_SOUS_RAYON,Hierarchy_FAMILLE,Hierarchy_SOUS_FAMILLE,Most_Purchased_Day_Category,Most_Purchased_Time_Of_Day,Quantity_per_Purchase,Revenue_per_Purchase,Revenue_per_Customer,Quantity_per_Customer,Average_Share_Of_Wallet,Avg_Purchase_Frequency
0,Carotte rapée 250 gr KG,34103,37512.00,43020.10,0.268018,0,CARD,High-Performance Fast-Moving Products,Core Revenue Drivers,"High Volume, Evening-Dominant, Weekday Preference",Mass-Market Hybrid Payment Products,High-Value & Promotion-Driven Products,Regional Best-Sellers,Fruits et légumes,Fraîche découpe,Prêt à assaisonner,FD - Crudité,Weekday,Afternoon,1.099962,1.261476,2.535217,2.210619,0.022722,2.009724
1,Légumes vapeur 500gr,10874,12032.64,36373.66,0.239088,153,CARD,High-Performance Fast-Moving Products,Core Revenue Drivers,"High Volume, Evening-Dominant, Weekday Preference",Mass-Market Hybrid Payment Products,High-Value & Promotion-Driven Products,Regional Best-Sellers,Fruits et légumes,Fraîche découpe,Prêt à cuire,FD - Poêlée et Wok,Weekday,Afternoon,1.106551,3.345012,5.924049,1.959713,0.035902,1.771010
2,poêlée Gourmande 525gr,293,313.00,1151.03,0.032129,0,CARD,"Moderate-Performance, Potential Growth Products","Low-Engagement, Marginal Products","Highest Volume, Morning-Dominant, Strong Weekd...",Consistent Card-Purchased Products,Moderate-Value Regular Essentials,Bron-Focused Leaders,Fruits et légumes,Fraîche découpe,Prêt à cuire,FD - Poêlée et Wok,Weekday,Afternoon,1.068259,3.928430,4.622610,1.257028,0.034042,1.176707
3,Champ. émincés 145 Gr,27437,31101.99,45834.79,0.269797,0,CARD,High-Performance Fast-Moving Products,Core Revenue Drivers,"High Volume, Evening-Dominant, Weekday Preference",Mass-Market Hybrid Payment Products,High-Value & Promotion-Driven Products,Regional Best-Sellers,Fruits et légumes,Fraîche découpe,Prêt à cuire,FD - Mono légume,Weekday,Afternoon,1.133578,1.670547,3.275785,2.222841,0.023131,1.960906
4,Champ. Eminces 530 Gr,25600,28428.00,95731.63,0.284402,0,CARD,High-Performance Fast-Moving Products,Core Revenue Drivers,"High Volume, Evening-Dominant, Weekday Preference",Mass-Market Hybrid Payment Products,High-Value & Promotion-Driven Products,Regional Best-Sellers,Fruits et légumes,Fraîche découpe,Prêt à cuire,FD - Mono légume,Weekday,Afternoon,1.110469,3.739517,7.330701,2.176890,0.043818,1.960334


In [97]:
label_details.head()

,Label,Description
0,High-Performance Fast-Moving Products,This group represents products characterized b...
1,"Slow-Moving, Low-Demand Products",This group shows characteristics of extremely ...
2,"Moderate-Performance, Potential Growth Products",This group comprises products positioned betwe...
3,"Highest Volume, Morning-Dominant, Strong Weekd...",This cluster has the highest mean and median f...
4,"Moderate Volume, Afternoon-Dominant, Weekday P...",This cluster shows moderate levels of Count_of...


In [98]:
segmentation_columns = [
    "sales_performance_segmentation",
    "customer_engagement_segmentation",
    "purchase_timing_segmentation",
    "payment_behavior_segmentation",
    "product_segmentation"  # Add more if needed
]

for col in segmentation_columns:
    # Rename for merge
    temp_details = label_details.rename(columns={"Label": col, "Description": "desc"})
    
    # Merge with product_labeled_df
    product_labeled_df = product_labeled_df.merge(temp_details, on=col, how="left")
    
    # Update the column in-place with "Label: Description"
    product_labeled_df[col] = product_labeled_df[col] + ": " + product_labeled_df["desc"]
    
    # Drop the temp desc column
    product_labeled_df.drop(columns=["desc"], inplace=True)


In [99]:
product_labeled_df.tail()

,Product_Name,Count_of_Purchases,Total_Quantity,Total_Revenue,Repeat_Purchase_Rate,Days_Since_Last_Purchase,most_used_payment_type,sales_performance_segmentation,customer_engagement_segmentation,purchase_timing_segmentation,payment_behavior_segmentation,product_segmentation,geographical_segmentation,Hierarchy_METIER,Hierarchy_SOUS_RAYON,Hierarchy_FAMILLE,Hierarchy_SOUS_FAMILLE,Most_Purchased_Day_Category,Most_Purchased_Time_Of_Day,Quantity_per_Purchase,Revenue_per_Purchase,Revenue_per_Customer,Quantity_per_Customer,Average_Share_Of_Wallet,Avg_Purchase_Frequency
3440,IT - Filets de maquereau fumés au poivre 200g,1,1.0,2.99,0.0,245,CARD,"Slow-Moving, Low-Demand Products: This group s...","Low-Engagement, Marginal Products: This cluste...","Highest Volume, Morning-Dominant, Strong Weekd...",Consistent Card-Purchased Products: These prod...,Luxury or Special-Occasion Items: This cluster...,Bron-Focused Leaders,Traiteur,Elaborés,Poissons fumés,Autres poissons fumés,Weekend,Morning,1.0,2.99,2.99,1.0,2.498120e-03,1.0
3441,IT - Guacamole 200g,1,1.0,2.55,0.0,259,CARD,"Slow-Moving, Low-Demand Products: This group s...",High-Value Niche Performers: This cluster has ...,"Low Volume, Balanced Timing, Mixed Weekday-Wee...",Consistent Card-Purchased Products: These prod...,Moderate-Value Regular Essentials: This group ...,Mions-Centric Products,Traiteur,Elaborés,Tartinables,Tarti Classiques,Weekend,Evening,1.0,2.55,2.55,1.0,1.000000e+00,1.0
3442,IT - Filet de truite fumée 160g,1,1.0,12.99,0.0,253,CARD,"Slow-Moving, Low-Demand Products: This group s...",High-Value Niche Performers: This cluster has ...,"Moderate Volume, Afternoon-Dominant, Weekday P...",Weekday-Focused Card Products: These products ...,Luxury or Special-Occasion Items: This cluster...,Mions-Centric Products,Traiteur,Elaborés,Poissons fumés,Truites fumées,Weekday,Afternoon,1.0,12.99,12.99,1.0,1.610623e-02,1.0
3443,IT - Saumon Gravelax,1,1.0,8.24,0.0,328,CARD,"Slow-Moving, Low-Demand Products: This group s...",High-Value Niche Performers: This cluster has ...,"Moderate Volume, Afternoon-Dominant, Weekday P...",Premium Card-Favored Products: These products ...,Card-Preferred Premium Purchases: This cluster...,Mions-Centric Products,Traiteur,Elaborés,Poissons fumés,Saumons fumés,Weekend,Afternoon,1.0,8.24,8.24,1.0,8.882182e-02,1.0
3444,IT -FIletto di Salmone alla mediterranea 300g pf,1,1.0,3.99,0.0,29,CASH,"Moderate-Performance, Potential Growth Product...","Low-Engagement, Marginal Products: This cluste...","Moderate Volume, Afternoon-Dominant, Weekday P...",Cash-Dominant Products: Products in this clust...,"Low-Value, Convenience-Oriented Products: This...",Vaulx-en-Velin Specialists,Traiteur,Elaborés,Piatti pronti,Cotture al forno,Weekday,Afternoon,1.0,3.99,3.99,1.0,4.533589e-07,1.0


In [100]:
# 1. Define segmentation columns for RAG input
rag_columns = [
    "Product_Name",
    "sales_performance_segmentation",
    "customer_engagement_segmentation",
    "purchase_timing_segmentation",
    "payment_behavior_segmentation",
    "product_segmentation",
    "geographical_segmentation"
]

# 2. Copy segmentation columns from product_labeled_df
rag_df = product_labeled_df[rag_columns].copy()


metric_columns = [
    "Count_of_Purchases",
    "Total_Quantity",
    "Total_Revenue",
    "Repeat_Purchase_Rate",
    "Avg_Purchase_Frequency",               
    "Average_Share_Of_Wallet",             
    "most_used_payment_type",
    "Most_Purchased_Day_Category",
    "Most_Purchased_Time_Of_Day",
    "Quantity_per_Purchase",
    "Revenue_per_Purchase",
    "Revenue_per_Customer",
    "Quantity_per_Customer"
]

# 4. Merge metrics into rag_df on 'Product_Name'
rag_df = rag_df.merge(
    product_labeled_df[["Product_Name"] + metric_columns],
    on="Product_Name",
    how="left"
)

In [101]:
rag_df.columns

Index(['Product_Name', 'sales_performance_segmentation', 'customer_engagement_segmentation', 'purchase_timing_segmentation', 'payment_behavior_segmentation', 'product_segmentation', 'geographical_segmentation', 'Count_of_Purchases', 'Total_Quantity', 'Total_Revenue', 'Repeat_Purchase_Rate', 'Avg_Purchase_Frequency', 'Average_Share_Of_Wallet', 'most_used_payment_type', 'Most_Purchased_Day_Category', 'Most_Purchased_Time_Of_Day', 'Quantity_per_Purchase', 'Revenue_per_Purchase', 'Revenue_per_Customer', 'Quantity_per_Customer'], dtype='object')

In [102]:
import re

def clean_product_name(name):
    # Remove symbols like !, *, :, etc. — keep letters, numbers, accents, spaces, dashes
    return re.sub(r"[^\w\s\-éèêàùçâäîïôöûüÉÈÊÀÙÇÂÄÎÏÔÖÛÜgG]", "", name).strip()

In [103]:
rag_df["Product_Name"] = rag_df["Product_Name"].apply(clean_product_name)


In [104]:


# 5. Build RAG input for LLM (enhanced version)
def build_rag_input(row):
    return f"""
Product: {row['Product_Name']}

--- Product Metrics Summary ---

Total Purchases: {row['Count_of_Purchases']}

Total Quantity Sold: {row['Total_Quantity']}

Total Revenue: €{row['Total_Revenue']:.2f}

Repeat Purchase Rate: {row['Repeat_Purchase_Rate']:.2%}

Average Purchase Frequency: {row['Avg_Purchase_Frequency']:.2f}

Average Share of Wallet: {row['Average_Share_Of_Wallet']:.2%}

Average Quantity per Purchase: {row['Quantity_per_Purchase']:.2f}

Average Revenue per Purchase: €{row['Revenue_per_Purchase']:.2f}

Average Quantity per Customer: {row['Quantity_per_Customer']:.2f}

Average Revenue per Customer: €{row['Revenue_per_Customer']:.2f}

Most Used Payment Type: {row['most_used_payment_type']}

Most Purchased Day Category: {row['Most_Purchased_Day_Category']}

Most Purchased Time of Day: {row['Most_Purchased_Time_Of_Day']}

--- Product Segmentation & Descriptions ---

Sales Performance: {row['sales_performance_segmentation']}

Customer Engagement: {row['customer_engagement_segmentation']}

Purchase Timing: {row['purchase_timing_segmentation']}

Payment Behavior: {row['payment_behavior_segmentation']}

Product Type: {row['product_segmentation']}

Geographical Segment: {row['geographical_segmentation']}

""".strip()

# 6. Apply the function to generate 'rag_input'
rag_df["rag_input"] = rag_df.apply(build_rag_input, axis=1)


In [105]:
rag_df.head(2)

,Product_Name,sales_performance_segmentation,customer_engagement_segmentation,purchase_timing_segmentation,payment_behavior_segmentation,product_segmentation,geographical_segmentation,Count_of_Purchases,Total_Quantity,Total_Revenue,Repeat_Purchase_Rate,Avg_Purchase_Frequency,Average_Share_Of_Wallet,most_used_payment_type,Most_Purchased_Day_Category,Most_Purchased_Time_Of_Day,Quantity_per_Purchase,Revenue_per_Purchase,Revenue_per_Customer,Quantity_per_Customer,rag_input
0,Carotte rapée 250 gr KG,High-Performance Fast-Moving Products: This gr...,Core Revenue Drivers: This cluster features th...,"High Volume, Evening-Dominant, Weekday Prefere...",Mass-Market Hybrid Payment Products: Products ...,High-Value & Promotion-Driven Products: This c...,Regional Best-Sellers,34103,37512.00,43020.10,0.268018,2.009724,0.022722,CARD,Weekday,Afternoon,1.099962,1.261476,2.535217,2.210619,Product: Carotte rapée 250 gr KG\n\n--- Produc...
1,Légumes vapeur 500gr,High-Performance Fast-Moving Products: This gr...,Core Revenue Drivers: This cluster features th...,"High Volume, Evening-Dominant, Weekday Prefere...",Mass-Market Hybrid Payment Products: Products ...,High-Value & Promotion-Driven Products: This c...,Regional Best-Sellers,10874,12032.64,36373.66,0.239088,1.771010,0.035902,CARD,Weekday,Afternoon,1.106551,3.345012,5.924049,1.959713,Product: Légumes vapeur 500gr\n\n--- Product M...


In [106]:
rag_df["rag_input"].iloc[0]

"Product: Carotte rapée 250 gr KG\n\n--- Product Metrics Summary ---\n\nTotal Purchases: 34103\n\nTotal Quantity Sold: 37512.0\n\nTotal Revenue: €43020.10\n\nRepeat Purchase Rate: 26.80%\n\nAverage Purchase Frequency: 2.01\n\nAverage Share of Wallet: 2.27%\n\nAverage Quantity per Purchase: 1.10\n\nAverage Revenue per Purchase: €1.26\n\nAverage Quantity per Customer: 2.21\n\nAverage Revenue per Customer: €2.54\n\nMost Used Payment Type: CARD\n\nMost Purchased Day Category: Weekday\n\nMost Purchased Time of Day: Afternoon\n\n--- Product Segmentation & Descriptions ---\n\nSales Performance: High-Performance Fast-Moving Products: This group represents products characterized by very high average revenue, large quantities sold, and frequent purchases (high 'Count of Purchases'). These products have minimal 'days since last purchase,' indicating they sell regularly and consistently. They also have significant engagement with promotional activities, evident from high values in both national an

In [107]:
from sentence_transformers import SentenceTransformer

# Load improved embedding model (better than MiniLM for RAG use cases)
model = SentenceTransformer("BAAI/bge-base-en-v1.5")

# Create normalized embeddings for cosine similarity (important for FAISS IP index)
embeddings = model.encode(
    rag_df["rag_input"].tolist(),
    normalize_embeddings=True,
    show_progress_bar=True
)


Batches: 100%|██████████| 111/111 [02:04<00:00,  1.12s/it]


In [108]:
import faiss
import numpy as np

# Normalize embeddings + convert to float32
embedding_matrix = np.array(embeddings).astype("float32")

# Use cosine similarity via inner product on normalized embeddings
index = faiss.IndexFlatIP(embedding_matrix.shape[1])
index.add(embedding_matrix)


In [109]:
# Save
faiss.write_index(index, "./Data/product_vectors.index")
print("✅ Saved updated index with dimension:", embedding_matrix.shape[1])

✅ Saved updated index with dimension: 768


In [110]:
rag_df.to_parquet("./Data/rag_df.parquet")


In [111]:
query = "Which products are underperforming in their target segments?"
query_vec = model.encode([query], normalize_embeddings=True).astype("float32")

# Search top-k matches
k = 3
D, I = index.search(query_vec, k)

# Display results
for i, idx in enumerate(I[0]):
    similarity = D[0][i]
    product_info = rag_df.iloc[idx]
    print(f"--- Result {i+1} ---")
    print(f"Similarity Score: {similarity:.4f}")
    print(f"Product Name: {product_info['Product_Name']}")
    print(f"Profile: {product_info['rag_input'][:500]}...")  # Preview
    print()


--- Result 1 ---
Similarity Score: 0.6568
Product Name: Crevette cuite 5060 ASC PV 400g
Profile: Product: Crevette cuite 5060 ASC PV 400g

--- Product Metrics Summary ---

Total Purchases: 3178

Total Quantity Sold: 3942.0

Total Revenue: €19001.46

Repeat Purchase Rate: 3.72%

Average Purchase Frequency: 1.26

Average Share of Wallet: 6.44%

Average Quantity per Purchase: 1.24

Average Revenue per Purchase: €5.98

Average Quantity per Customer: 1.56

Average Revenue per Customer: €7.52

Most Used Payment Type: CARD

Most Purchased Day Category: Weekday

Most Purchased Time of Day: Afternoo...

--- Result 2 ---
Similarity Score: 0.6532
Product Name: Crevette cuite 5060 ASC PV 320g
Profile: Product: Crevette cuite 5060 ASC PV 320g

--- Product Metrics Summary ---

Total Purchases: 1108

Total Quantity Sold: 1329.0

Total Revenue: €4803.43

Repeat Purchase Rate: 0.87%

Average Purchase Frequency: 1.21

Average Share of Wallet: 4.03%

Average Quantity per Purchase: 1.20

Average Revenue p

In [112]:
query = "Which products need better product strategies?" 
query_vec = model.encode([query], normalize_embeddings=True).astype("float32")  # Normalization is important!

# Search top 3 similar products
k = 3
D, I = index.search(query_vec, k)

# Display retrieved product profiles
for i, idx in enumerate(I[0]):
    product_info = rag_df.iloc[idx]
    similarity = D[0][i]
    
    print(f"--- Result {i+1} ---")
    print(f"Similarity Score: {similarity:.4f}")
    print(f"Product Name: {product_info['Product_Name']}")
    print(f"Profile: {product_info['rag_input'][:500]}...")  # Limit text for readability
    print()


--- Result 1 ---
Similarity Score: 0.6158
Product Name: Tomme Savoie PE PMALIN
Profile: Product: Tomme Savoie PE PMALIN

--- Product Metrics Summary ---

Total Purchases: 2

Total Quantity Sold: 2.0

Total Revenue: €5.30

Repeat Purchase Rate: 0.00%

Average Purchase Frequency: 1.00

Average Share of Wallet: 0.21%

Average Quantity per Purchase: 1.00

Average Revenue per Purchase: €2.65

Average Quantity per Customer: 1.00

Average Revenue per Customer: €2.65

Most Used Payment Type: CARD

Most Purchased Day Category: Weekday

Most Purchased Time of Day: Afternoon

--- Product Segm...

--- Result 2 ---
Similarity Score: 0.6147
Product Name: Menthe Pot HVE
Profile: Product: Menthe Pot HVE

--- Product Metrics Summary ---

Total Purchases: 3155

Total Quantity Sold: 3823.0

Total Revenue: €7182.01

Repeat Purchase Rate: 5.57%

Average Purchase Frequency: 1.27

Average Share of Wallet: 2.95%

Average Quantity per Purchase: 1.21

Average Revenue per Purchase: €2.28

Average Quantity per Cu

In [113]:

# --- Set similarity threshold ---
SIMILARITY_THRESHOLD_TEST = 0.5  # You can adjust this

# --- Test queries ---
test_queries = [
    "Which products need better product strategies?",
    "Find items with high churn rate and low retention.",
    "Show top-selling products.",
    "Bad product?",
    "Improve conversion performance",
    "Which products to bundle together?",
    "List expensive items with low sales.",
    "Need retention ideas."
]

# --- Run similarity search for each query ---
for query in test_queries:
    print("=" * 80)
    print(f"🔎 Query: {query}")
    query_vec = model.encode([query], normalize_embeddings=True).astype("float32")
    D, I = index.search(query_vec, k=3)

    if D[0][0] >= SIMILARITY_THRESHOLD_TEST:
        print(f"✅ Top Match Passed Threshold: {D[0][0]:.4f} >= {SIMILARITY_THRESHOLD_TEST}")
        for i, idx in enumerate(I[0]):
            product_info = rag_df.iloc[idx]
            similarity = D[0][i]
            print(f"\n--- Result {i+1} ---")
            print(f"Similarity Score: {similarity:.4f}")
            print(f"Product Name: {product_info['Product_Name']}")
            print(f"Profile Preview: {product_info['rag_input'][:300]}...")
    else:
        print(f"❌ No strong product match (Top score: {D[0][0]:.4f} < {SIMILARITY_THRESHOLD_TEST})")
        print("💡 Suggest the user to rephrase the query.\n")

🔎 Query: Which products need better product strategies?
✅ Top Match Passed Threshold: 0.6158 >= 0.5

--- Result 1 ---
Similarity Score: 0.6158
Product Name: Tomme Savoie PE PMALIN
Profile Preview: Product: Tomme Savoie PE PMALIN

--- Product Metrics Summary ---

Total Purchases: 2

Total Quantity Sold: 2.0

Total Revenue: €5.30

Repeat Purchase Rate: 0.00%

Average Purchase Frequency: 1.00

Average Share of Wallet: 0.21%

Average Quantity per Purchase: 1.00

Average Revenue per Purchase: €2.6...

--- Result 2 ---
Similarity Score: 0.6147
Product Name: Menthe Pot HVE
Profile Preview: Product: Menthe Pot HVE

--- Product Metrics Summary ---

Total Purchases: 3155

Total Quantity Sold: 3823.0

Total Revenue: €7182.01

Repeat Purchase Rate: 5.57%

Average Purchase Frequency: 1.27

Average Share of Wallet: 2.95%

Average Quantity per Purchase: 1.21

Average Revenue per Purchase: €2....

--- Result 3 ---
Similarity Score: 0.6134
Product Name: Graines germées betterave
Profile Preview: Produc

In [114]:
results = rag_df.iloc[I[0]]["rag_input"].tolist()


In [115]:
results

["Product: Houmous 175g fra\n\n--- Product Metrics Summary ---\n\nTotal Purchases: 4180\n\nTotal Quantity Sold: 4339.0\n\nTotal Revenue: €11238.01\n\nRepeat Purchase Rate: 13.85%\n\nAverage Purchase Frequency: 1.33\n\nAverage Share of Wallet: 3.62%\n\nAverage Quantity per Purchase: 1.04\n\nAverage Revenue per Purchase: €2.69\n\nAverage Quantity per Customer: 1.38\n\nAverage Revenue per Customer: €3.57\n\nMost Used Payment Type: CARD\n\nMost Purchased Day Category: Weekday\n\nMost Purchased Time of Day: Afternoon\n\n--- Product Segmentation & Descriptions ---\n\nSales Performance: Moderate-Performance, Potential Growth Products: This group comprises products positioned between high-performance and slow-moving categories. They exhibit moderate revenue and quantities sold, with occasional and steady purchase frequency. Their 'days since last purchase' metric is lower than the slow-moving group, indicating moderate customer interest and consistent replenishment, though not as frequently as